In [1]:
from instagrapi import Client
from instagrapi.exceptions import LoginRequired
import toml
import logging

import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from PIL import Image
import requests
from io import BytesIO

import pytesseract
import malaya

/home/faiz/anaconda3/envs/work/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/faiz/anaconda3/envs/work/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


### Initialise client and load secrets file

In [2]:
cl = Client()

with open('secrets.toml', 'r') as f:
    secrets = toml.load(f)

### Login management

#### Create session file
Skip this if `session.json` has been created.

In [ ]:

cl.login(secrets["instagram"]["username"], secrets["instagram"]["password"])

# save session file
cl.dump_settings("session.json")

### Login function

In [4]:
logger = logging.getLogger()


def login_user():
    """
    Attempts to login to Instagram using either the provided session information
    or the provided username and password.
    """

    session = cl.load_settings("session.json")

    login_via_session = False
    login_via_pw = False

    if session:
        try:
            cl.set_settings(session)
            cl.login(secrets["instagram"]["username"], secrets["instagram"]["password"])

            # check if session is valid
            try:
                cl.get_timeline_feed()
            except LoginRequired:
                logger.info("Session is invalid, need to login via username and password")

                old_session = cl.get_settings()

                # use the same device uuids across logins
                cl.set_settings({})
                cl.set_uuids(old_session["uuids"])

                cl.login(secrets["instagram"]["username"], secrets["instagram"]["password"])
            login_via_session = True
        except Exception as e:
            logger.info("Couldn't login user using session information: %s" % e)

    if not login_via_session:
        try:
            logger.info("Attempting to login via username and password. username: %s" % secrets["instagram"]["username"])
            if cl.login(secrets["instagram"]["username"], secrets["instagram"]["password"]):
                login_via_pw = True
        except Exception as e:
            logger.info("Couldn't login user using username and password: %s" % e)

    if not login_via_pw and not login_via_session:
        raise Exception("Couldn't login user with either password or session")

#### Login

In [5]:
login_user()

In [15]:
# hashtag_dump = cl.hashtag_medias_recent_v1("UniKL", 50)
hashtag_dump = cl.hashtag_medias_recent("UniKL", 50)

In [16]:
tmp = []
for i in range(0,50):
    tmp.append(hashtag_dump[i].model_dump())

hashtag_posts = pd.DataFrame(tmp)

In [ ]:
hashtag_posts

### Plotting location on the map 

In [17]:
def media_type_func(x,y):
    if x == 1:
        return "Photo"
    elif x == 2 and y == "feed":
        return "Video"
    elif x == 2 and y == "igtv":
        return "IGTV"
    elif x == 2 and y == "clips":
        return "Reel"
    else:
        return "Album"

In [18]:
geo_df = pd.DataFrame()

geo_df["User"] = hashtag_posts["user"].str.get("username")
geo_df["Taken at"] = hashtag_posts["taken_at"]
geo_df["Type"] = hashtag_posts["media_type"]
geo_df["Product type"] = hashtag_posts["product_type"]

geo_df["Type"] = geo_df.apply(lambda x: media_type_func(x["Type"], x["Product type"]), axis=1)

geo_df["Lat"] = hashtag_posts["location"].str.get("lat")
geo_df["Lan"] = hashtag_posts["location"].str.get("lng")

In [19]:
fig = px.scatter_map(geo_df, lat="Lat", lon="Lan", map_style="carto-darkmatter", hover_name="User", hover_data="Taken at", color="Type", zoom=11)
fig.show()

### Text preprocessing

In [20]:
# Unix/Linux
tesseract_path = r"/usr/bin/tesseract"

# Set tesseract path
pytesseract.pytesseract.tesseract_cmd = tesseract_path


#### Extract text from photo

In [21]:
img_texts = []

for img_url in hashtag_posts["thumbnail_url"]:
    if img_url:
        response = requests.get(img_url)
        img_text = pytesseract.image_to_string(Image.open(BytesIO(response.content)))
        print(f"Extracted text: {img_text}")
        img_texts.append(img_text)
    else:
        img_texts.append("")

hashtag_posts["thumbnail_text"] = img_texts

Extracted text: a


Extracted text: 
Extracted text: 
Extracted text: 
Extracted text: CK
UNIVERSITI
KUALA LUMPUR

YOUR

TRIAL SPM
RESULTS
ARE ALL YOU
NEED TO
APPLY!

SPM 2024 students
can now apply for
the July 2025
intake using

Trial SPM results.

© »Ppty Topay:


Extracted text: 
Extracted text: Salam dan selamat petang..bagaimana
feedback tentang kerepek kami? @
4:55PM YW

WsIm 4:57pm

Alhamdulilah 4:53 py)

Kerepek slmt smpai, seller fast respon
4:58PM

Thanks tauu review sy bagi a-sgpyy

100000/100_,

@© swackFoRtire @© swack4viree

Extracted text: aaa ie eRe
VAS SINGH BHADAURIA

This is suko


Extracted text: PLEASE
f ROTATE
YOUR PHONE

Extracted text: 
Extracted text: 
Extracted text: 
Extracted text: © \ DR ASYRAF WAJDI DUSUKI
Posted by Asyraf Wajdi Dusuki
3h:-@

Ada minta saya pantau isu di
UITM dan jangan hanya fokus
MRSM. UiTM bukan lagi di
bawah MARA sejak diambil alih
KPT 1 Jun 1999. Di bawah MARA
ada 344 IPMA cth: UniKL, UPTM,
IKM, KPTM, KKTM, KM, KPM &
GiatMARA.

Extra

### Preprocessing

* Normalisation
* Segmentation
* Demoji

In [79]:
def preprocess_text(text):
    # this uses NLTK
    # tokens = word_tokenize(text.lower())

    # malaya segmentation
    # segmenter = malaya.segmentation.huggingface()
    # segmenter_func = lambda x: segmenter.generate([x], max_length = 100)[0]

    # demoji
    demoji = malaya.preprocessing.demoji()
    demoji_text = demoji.demoji(text)

    # malaya preproc
    preproc = malaya.preprocessing.preprocessing(demoji=demoji)
    preproc_text = "".join(preproc.process(text))

    for k,v in demoji_text.items():
        preproc_text = preproc_text.replace(k,v)

    w_tokeniser = malaya.tokenizer.Tokenizer()
    s_tokeniser = malaya.tokenizer.SentenceTokenizer()

    w_tokens = w_tokeniser.tokenize(preproc_text, lowercase=True)
    # s_tokens = s_tokeniser.tokenize(preproc_text)

    # remove alphanumeric word(s)
    # filtered_tokens = [word for word in tokens if word.isalnum()]
    
    return ' '.join(w_tokens)

    # return preproc_text

In [ ]:
hashtag_posts

In [78]:
texts = []
for i, caption in enumerate(hashtag_posts["caption_text"]):
    if hashtag_posts.loc[i, "thumbnail_text"] == "":
        texts.append(preprocess_text(caption))
    else:
        text = caption + hashtag_posts.loc[i, "thumbnail_text"]
        texts.append(preprocess_text(text))

texts

['pagiinimengadapsupervisor <repeated> . </repeated> prof . dr. sulaimansajilan <allcaps> deanuniversitikualalumpurbusinessschool </allcaps> <hashtag> #unikl </hashtag> <hashtag> #phdstudent </hashtag> <hashtag> #study </hashtag> <hashtag> #workhardplayhard </hashtag> <hashtag> #worksmartstudysmart </hashtag> <hashtag> #studenta </hashtag>',
 '<number> march <number> manager <user> supervisor <user> secretary <user> . kin_19 <user> . pelajar <hashtag> #unikl </hashtag> <hashtag> #likeforlikes </hashtag> <hashtag> #landscape </hashtag> <hashtag> #lawakhambar </hashtag> <hashtag> #physics </hashtag> <hashtag> #lawaktahapdewa </hashtag> <hashtag> #lawak </hashtag> <hashtag> #sukasukaje </hashtag> <hashtag> #love </hashtag> <hashtag> #instagood </hashtag> <hashtag> #photooftheday </hashtag> <hashtag> #beautiful </hashtag> <hashtag> #happy </hashtag> <hashtag> #cute </hashtag> <hashtag> #tbt </hashtag> <hashtag> #like4like </hashtag> <hashtag> #follow4follow </hashtag> <hashtag> #art </hash

#### Detect language

In [69]:
import operator
fast_text = malaya.language_detection.fasttext()

langs = []
lang_probas = []

for text in texts:
    prob_dict = fast_text.predict_proba([text])
    print(prob_dict)

    lang = max(prob_dict[0].items(), key=operator.itemgetter(1))[0]
    prob = max(prob_dict[0].items(), key=operator.itemgetter(1))[1]
    
    langs.append(lang)
    lang_probas.append(prob)

hashtag_posts["Language"] = langs
hashtag_posts["Language Probability"] = lang_probas

[{'standard-english': 0.9945177, 'local-english': 0.0, 'manglish': 0.0, 'standard-indonesian': 0.0, 'socialmedia-indonesian': 0.0, 'standard-malay': 0.0, 'local-malay': 0.0, 'standard-mandarin': 0.0, 'local-mandarin': 0.0, 'other': 0.0}]
[{'standard-english': 0.9716635, 'local-english': 0.0, 'manglish': 0.0, 'standard-indonesian': 0.0, 'socialmedia-indonesian': 0.0, 'standard-malay': 0.0, 'local-malay': 0.0, 'standard-mandarin': 0.0, 'local-mandarin': 0.0, 'other': 0.0}]
[{'standard-english': 0.0, 'local-english': 0.0, 'manglish': 0.0, 'standard-indonesian': 0.0, 'socialmedia-indonesian': 0.0, 'standard-malay': 0.9414176, 'local-malay': 0.0, 'standard-mandarin': 0.0, 'local-mandarin': 0.0, 'other': 0.0}]
[{'standard-english': 0.0, 'local-english': 0.0, 'manglish': 0.0, 'standard-indonesian': 0.0, 'socialmedia-indonesian': 0.0, 'standard-malay': 0.0, 'local-malay': 0.0, 'standard-mandarin': 0.0, 'local-mandarin': 0.5823944, 'other': 0.0}]
[{'standard-english': 0.0, 'local-english': 0.75

### Sentiment Analysis

In [28]:
malaya.sentiment.available_huggingface

{'mesolitica/sentiment-analysis-nanot5-tiny-malaysian-cased': {'Size (MB)': 93,
  'macro precision': 0.67768,
  'macro recall': 0.68266,
  'macro f1-score': 0.67997},
 'mesolitica/sentiment-analysis-nanot5-small-malaysian-cased': {'Size (MB)': 167,
  'macro precision': 0.67602,
  'macro recall': 0.6712,
  'macro f1-score': 0.67339}}

In [30]:
print(malaya.sentiment.info)

Trained on https://huggingface.co/datasets/mesolitica/chatgpt-explain-sentiment
Split 90% to train, 10% to test.


In [41]:
# load malaya sentiment model from Huggingface
sent_model = malaya.sentiment.huggingface(model="mesolitica/sentiment-analysis-nanot5-small-malaysian-cased")

In [80]:
sentiments = []
sentiment_probas = []
for i, text in enumerate(texts):
    if text:
        prob_dict = sent_model.predict_proba([text])
        sentiment = max(prob_dict[0].items(), key=operator.itemgetter(1))[0]
        sentiment_prob = max(prob_dict[0].items(), key=operator.itemgetter(1))[1]
    else:
        sentiment = "neutral"
        sentiment_prob = 0.000
    
    sentiments.append(sentiment)
    sentiment_probas.append(sentiment_prob)

hashtag_posts["sentiment"] = sentiments
hashtag_posts["sentiment probability"] = sentiment_probas

### Emotion analysis

In [71]:
malaya.emotion.available_huggingface

{'mesolitica/emotion-analysis-nanot5-tiny-malaysian-cased': {'Size (MB)': 93,
  'macro precision': 0.96107,
  'macro recall': 0.9627,
  'macro f1-score': 0.96182},
 'mesolitica/emotion-analysis-nanot5-small-malaysian-cased': {'Size (MB)': 167,
  'macro precision': 0.96814,
  'macro recall': 0.97004,
  'macro f1-score': 0.96905}}

In [56]:
print(malaya.emotion.info)

Trained on https://github.com/huseinzol05/malaysian-dataset/tree/master/corpus/emotion
Split 80% to train, 20% to test.


In [57]:
# load malaya emotion model from Huggingface
emo_model = malaya.emotion.huggingface(model="mesolitica/emotion-analysis-nanot5-small-malaysian-cased")

In [81]:
emotions = []
emotion_probas = []
for i, text in enumerate(texts):
    if text:
        prob_dict = emo_model.predict_proba([text])
        emotion = max(prob_dict[0].items(), key=operator.itemgetter(1))[0]
        emotion_prob = max(prob_dict[0].items(), key=operator.itemgetter(1))[1]
    else:
        emotion = "no emotion"
        emotion_prob = 0.000
    
    emotions.append(emotion)
    emotion_probas.append(emotion_prob)

hashtag_posts["emotion"] = emotions
hashtag_posts["emotion probability"] = emotion_probas